In [4]:
import os
import boto3
import requests
import pandas as pd
import pyarrow as pa
from io import BytesIO
import pyarrow.parquet as pq
from datetime import datetime

In [5]:
API_LINK = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [6]:
requisicao = requests.get(API_LINK)

In [7]:
wheater_json = requisicao.json()

In [8]:
weather_df = pd.json_normalize(wheater_json)

In [9]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1703694954,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1011,2.64,125,3.25,0,2,2020383,BR,1703665446,1703715994


In [10]:
df_lon_lat = pd.DataFrame(wheater_json['coord'], index=[0])

In [11]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [12]:
df_weather = pd.DataFrame(wheater_json['weather'], index=[0])

In [13]:
df_weather

,id,main,description,icon
0,800,Clear,clear sky,01d


In [14]:
df_main = pd.DataFrame(wheater_json['main'], index=[0])

In [15]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,300.17,300.91,299.26,303.95,1015,55,1015,1011


In [16]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [17]:
weather_df = pd.DataFrame(weather_df)

In [18]:
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1703694954,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1011,2.64,125,3.25,0,2,2020383,BR,1703665446,1703715994


In [19]:
df_clumns_selected = weather_df[selected_columns]

In [20]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1703694954,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,300.17,...,1011,2.64,125,3.25,0,2,2020383,BR,1703665446,1703715994


In [21]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [22]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,800.0,Clear,clear sky,01d,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,300.17,300.91,299.26,303.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1011.0,2.64,125.0,3.25,0.0,2.0,2020383.0,BR,1.703665e+09,1.703716e+09


In [23]:
super_df.to_parquet("C:/Users/Cristian/Desktop/area_de_trabalho/super_df.parquet")

Save at S3

In [24]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [25]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [26]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [27]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [28]:
dia_da_exucucao

'2023-12-27-13-37-11'

In [29]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [30]:
s3_file_name

'df_teste_2023-12-27-13-37-11.parquet'

In [31]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:/Users/crist/Desktop/s3_testes/super_df.parquet'

---------------------------------

CATUABA